In [2]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd

In [3]:
api_key = "Jc2gL6jUt0KRlsqfkKJ9kBHcP7m9IBB5M0ph7WV9AybJ6AupPEe3rxDgCWZmJcuH"
secret_key = pd.read_table("paper_secret_key.txt").columns[0]

In [4]:
client = Client(api_key=api_key, api_secret=secret_key, tld="com", testnet=True)

In [5]:
client

In [6]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1685186173220,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.10000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '7328.99298317', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

### 실시간 데이터 스트리밍과 수집

In [7]:
twm = ThreadedWebsocketManager()
twm.start()

In [8]:
df = pd.DataFrame(columns=["Open", "High", "Low", "Close", "Volume", "Complete"])
df

,Open,High,Low,Close,Volume,Complete


In [9]:
def stream_candles(msg):
    event_time = pd.to_datetime(msg["E"], unit="ms")
    start_time = pd.to_datetime(msg["k"]["t"], unit="ms")
    first = float(msg["k"]["o"])
    high = float(msg["k"]["h"])
    low = float(msg["k"]["l"])
    close = float(msg["k"]["c"])
    volume = float(msg["k"]["v"])
    complete = msg["k"]["x"]
    
    print("Time: {} | Price: {}".format(event_time, close))
    
    df.loc[start_time] = [first, high, low, close, volume, complete]

In [10]:
twm.start_kline_socket(callback=stream_candles, symbol="BTCUSDT", interval="1m")

'btcusdt@kline_1m'

Time: 2023-05-31 08:44:50.331000 | Price: 27117.58
Time: 2023-05-31 08:44:52.865000 | Price: 27117.58
Time: 2023-05-31 08:44:55.161000 | Price: 27117.57
Time: 2023-05-31 08:44:57.470000 | Price: 27117.58
Time: 2023-05-31 08:44:59.513000 | Price: 27117.58
Time: 2023-05-31 08:45:00.003000 | Price: 27117.57
Time: 2023-05-31 08:45:02.060000 | Price: 27117.58
Time: 2023-05-31 08:45:04.061000 | Price: 27115.6
Time: 2023-05-31 08:45:06.383000 | Price: 27115.59
Time: 2023-05-31 08:45:08.665000 | Price: 27115.6
Time: 2023-05-31 08:45:10.828000 | Price: 27115.6
Time: 2023-05-31 08:45:12.959000 | Price: 27115.59
Time: 2023-05-31 08:45:14.971000 | Price: 27115.59
Time: 2023-05-31 08:45:17.241000 | Price: 27113.48
Time: 2023-05-31 08:45:19.336000 | Price: 27113.48
Time: 2023-05-31 08:45:21.473000 | Price: 27113.49
Time: 2023-05-31 08:45:23.560000 | Price: 27122.64
Time: 2023-05-31 08:45:26.163000 | Price: 27122.64
Time: 2023-05-31 08:45:28.673000 | Price: 27122.65
Time: 2023-05-31 08:45:31.163000 |

In [11]:
twm.stop()

CANCEL read_loop


Time: 2023-05-31 08:46:30.167000 | Price: 27126.35


In [12]:
df

,Open,High,Low,Close,Volume,Complete
2023-05-31 08:44:00,27122.65,27122.66,27117.57,27117.57,23.67802,True
2023-05-31 08:45:00,27117.57,27123.00,27113.48,27122.99,43.63989,True
2023-05-31 08:46:00,27122.99,27129.00,27122.99,27126.35,44.16716,False


### 롱 전용 거래 클래스

In [20]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        self.data = pd.DataFrame(columns=["Open", "High", "Low", "Close", "Volume", "Complete"])
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        
    def start_trading(self):
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
            
    def stream_candles(self, msg):
        event_time = pd.to_datetime(msg["E"], unit="ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit="ms")
        first = float(msg["k"]["o"])
        high = float(msg["k"]["h"])
        low = float(msg["k"]["l"])
        close = float(msg["k"]["c"])
        volume = float(msg["k"]["v"])
        complete = msg["k"]["x"]

        print("Time: {} | Price: {}".format(event_time, close))

        self.data.loc[start_time] = [first, high, low, close, volume, complete]


In [21]:
trader = LongOnlyTrader(symbol="BTCUSDT", bar_length="1m")
trader

In [22]:
trader.symbol

'BTCUSDT'

In [23]:
trader.data

,Open,High,Low,Close,Volume,Complete


In [24]:
trader.available_intervals

['1m',
 '3m',
 '5m',
 '15m',
 '30m',
 '1h',
 '2h',
 '4h',
 '6h',
 '8h',
 '12h',
 '1d',
 '3d',
 '1w',
 '1M']

In [25]:
trader.start_trading()

Time: 2023-05-31 08:55:21.420000 | Price: 27115.0
Time: 2023-05-31 08:55:23.543000 | Price: 27115.0
Time: 2023-05-31 08:55:25.683000 | Price: 27115.0
Time: 2023-05-31 08:55:27.693000 | Price: 27111.94
Time: 2023-05-31 08:55:29.759000 | Price: 27117.98
Time: 2023-05-31 08:55:31.961000 | Price: 27119.07
Time: 2023-05-31 08:55:34.201000 | Price: 27120.49
Time: 2023-05-31 08:55:36.212000 | Price: 27120.49
Time: 2023-05-31 08:55:38.562000 | Price: 27120.49


In [26]:
trader.twm.stop()

CANCEL read_loop


Time: 2023-05-31 08:55:57.828000 | Price: 27122.78


In [27]:
trader.data

,Open,High,Low,Close,Volume,Complete
2023-05-31 08:55:00,27115.0,27123.02,27107.62,27122.78,113.94097,False


## 역대 데이터와 실시간 데이터 사용하기

* 역대 데이터를 가져오고 라이브 데이터와 합치기

In [29]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
from datetime import datetime, timedelta

In [30]:
client = Client(api_key=api_key, api_secret=secret_key, tld="com", testnet=True)

In [31]:
now = datetime.utcnow()
now

datetime.datetime(2023, 5, 31, 8, 59, 23, 202427)

In [32]:
past = now - timedelta(days=2)
past

datetime.datetime(2023, 5, 29, 8, 59, 23, 202427)

In [33]:
str(past)

'2023-05-29 08:59:23.202427'

In [34]:
def get_most_recent(symbol, interval, days):
    now = datetime.utcnow()
    past = str(now - timedelta(days))
    
    bars = client.get_historical_klines(symbol, interval, start_str=past, end_str=None, limit=1000)
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:, 0], unit="ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                  "Clos Time", "Quote Asset Volume", "Number of Trades",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace=True)
    
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors="coerce")
    df["Complete"] = [True for row in range(len(df) - 1)] + [False]
    
    return df

In [35]:
df = get_most_recent(symbol="BTCUSDT", interval="1m", days=2)
df

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2023-05-29 09:08:00,27890.09,27890.18,27888.70,27888.70,0.695218,True
2023-05-29 09:09:00,27888.70,27888.96,27887.39,27887.39,0.388649,True
2023-05-29 09:10:00,27887.40,27887.41,27887.35,27887.40,0.260405,True
2023-05-29 09:11:00,27887.39,27887.40,27887.38,27887.40,0.130821,True
2023-05-29 09:12:00,27887.40,27887.60,27886.48,27886.50,1.797094,True
...,...,...,...,...,...,...
2023-05-31 09:03:00,27121.28,27122.12,27121.28,27122.09,0.539660,True
2023-05-31 09:04:00,27122.09,27122.63,27122.09,27122.42,0.825832,True
2023-05-31 09:05:00,27122.41,27125.73,27122.41,27125.72,1.475886,True


In [36]:
get_most_recent(symbol="BTCUSDT", interval="1m", days=2/24)

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2023-05-31 07:09:00,27146.17,27146.18,27135.02,27135.02,1.583636,True
2023-05-31 07:10:00,27135.03,27141.21,27126.12,27141.21,5.057568,True
2023-05-31 07:11:00,27141.21,27141.24,27128.44,27128.70,1.654178,True
2023-05-31 07:12:00,27128.67,27128.68,27120.52,27123.66,1.891819,True
2023-05-31 07:13:00,27123.67,27126.71,27123.56,27126.71,1.454455,True
...,...,...,...,...,...,...
2023-05-31 09:04:00,27122.09,27122.63,27122.09,27122.42,0.825832,True
2023-05-31 09:05:00,27122.41,27125.73,27122.41,27125.72,1.475886,True
2023-05-31 09:06:00,27125.73,27125.77,27122.10,27125.77,2.678243,True


In [37]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
#         self.data = pd.DataFrame(columns=["Open", "High", "Low", "Close", "Volume", "Complete"])
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(self.symbol, self.bar_length, historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        
            
            
    def get_most_recent(self, symbol, interval, days):
        now = datetime.utcnow()
        past = str(now - timedelta(days))

        bars = client.get_historical_klines(symbol, interval, start_str=past, end_str=None, limit=1000)
        
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:, 0], unit="ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace=True)

        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors="coerce")
        df["Complete"] = [True for row in range(len(df) - 1)] + [False]
        
        self.data = df
            
    def stream_candles(self, msg):
        event_time = pd.to_datetime(msg["E"], unit="ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit="ms")
        first = float(msg["k"]["o"])
        high = float(msg["k"]["h"])
        low = float(msg["k"]["l"])
        close = float(msg["k"]["c"])
        volume = float(msg["k"]["v"])
        complete = msg["k"]["x"]

        print("Time: {} | Price: {}".format(event_time, close))

        self.data.loc[start_time] = [first, high, low, close, volume, complete]


In [38]:
trader = LongOnlyTrader(symbol="BTCUSDT", bar_length="1m")
trader

In [42]:
trader.start_trading(historical_days=2)

Time: 2023-05-31 10:37:54.779000 | Price: 27084.66
Time: 2023-05-31 10:37:57.425000 | Price: 27084.67
Time: 2023-05-31 10:37:59.625000 | Price: 27084.67
Time: 2023-05-31 10:38:00.003000 | Price: 27084.66
Time: 2023-05-31 10:38:02.108000 | Price: 27084.67
Time: 2023-05-31 10:38:04.587000 | Price: 27084.66


In [43]:
trader.twm.stop()

In [44]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2023-05-29 10:38:00,27900.89,27911.92,27900.88,27909.10,2.945461,True
2023-05-29 10:39:00,27908.97,27911.32,27908.66,27911.22,1.360004,True
2023-05-29 10:40:00,27911.22,27911.91,27911.05,27911.91,0.717536,True
2023-05-29 10:41:00,27910.80,27912.20,27904.84,27904.84,2.983951,True
2023-05-29 10:42:00,27904.84,27904.84,27901.36,27901.36,2.178509,True
...,...,...,...,...,...,...
2023-05-31 10:34:00,27068.29,27075.58,27068.10,27075.58,2.940977,True
2023-05-31 10:35:00,27075.58,27076.77,27075.57,27076.76,0.659767,True
2023-05-31 10:36:00,27076.76,27079.38,27076.73,27079.38,2.036265,True


CANCEL read_loop


Time: 2023-05-31 10:38:06.687000 | Price: 27084.67


## 롱 전용 전략 추가

트레이딩 봇 구현과 작동 확인을 위해 많은 거래 기회를 포착할 수 있는 파라미터 설정

* bar_length = "1m"
* 수익 임계값 = 0%
* Low, High 거래량 임계값 = -3, 3

In [55]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
pd.options.display.float_format = '{:.6f}'.format

In [56]:
client = Client(api_key=api_key, api_secret=secret_key, tld="com", testnet = True)

In [57]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh):
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(self.symbol, self.bar_length, historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        
            
            
    def get_most_recent(self, symbol, interval, days):
        now = datetime.utcnow()
        past = str(now - timedelta(days))

        bars = client.get_historical_klines(symbol, interval, start_str=past, end_str=None, limit=1000)
        
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:, 0], unit="ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace=True)

        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors="coerce")
        df["Complete"] = [True for row in range(len(df) - 1)] + [False]
        
        self.data = df
            
    def stream_candles(self, msg):
        event_time = pd.to_datetime(msg["E"], unit="ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit="ms")
        first = float(msg["k"]["o"])
        high = float(msg["k"]["h"])
        low = float(msg["k"]["l"])
        close = float(msg["k"]["c"])
        volume = float(msg["k"]["v"])
        complete = msg["k"]["x"]

        print(".", end = "", flush = True)

        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        if complete == True:
            self.define_strategy()
            
    def define_strategy(self):
        
        df = self.data.copy()
        
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        
        self.prepared_data = df.copy()


In [58]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]

In [59]:
trader = LongOnlyTrader(symbol, bar_length, return_thresh, volume_thresh)

In [64]:
trader.start_trading(historical_days=1/24)

....................................

In [65]:
trader.twm.stop()

In [66]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2023-05-31 10:02:00,27131.360000,27135.330000,27127.960000,27135.330000,2.589770,True
2023-05-31 10:03:00,27135.000000,27135.440000,27129.510000,27129.510000,2.062962,True
2023-05-31 10:04:00,27129.510000,27129.520000,27129.430000,27129.510000,0.232853,True
2023-05-31 10:05:00,27129.520000,27129.590000,27129.500000,27129.510000,0.231224,True
2023-05-31 10:06:00,27129.520000,27135.870000,27128.280000,27135.870000,2.131111,True
...,...,...,...,...,...,...
2023-05-31 10:59:00,27058.660000,27058.740000,27056.520000,27056.520000,0.402899,True
2023-05-31 11:00:00,27056.520000,27056.590000,27047.670000,27047.690000,3.255723,True
2023-05-31 11:01:00,27047.680000,27060.810000,27047.670000,27060.810000,3.033341,False


In [71]:
trader.prepared_data

,Close,Volume,returns,vol_ch,position
Date,,,,,
2023-05-31 10:02:00,27135.330000,2.589770,NaN,NaN,1
2023-05-31 10:03:00,27129.510000,2.062962,-0.000215,-0.227426,1
2023-05-31 10:04:00,27129.510000,0.232853,0.000000,-2.181491,0
2023-05-31 10:05:00,27129.510000,0.231224,0.000000,-0.007020,0
2023-05-31 10:06:00,27135.870000,2.131111,0.000234,2.221012,0
2023-05-31 10:07:00,27137.470000,1.283659,0.000059,-0.506929,0
2023-05-31 10:08:00,27128.410000,2.273817,-0.000334,0.571745,1
2023-05-31 10:09:00,27128.440000,0.149385,0.000001,-2.722688,0
2023-05-31 10:10:00,27138.490000,2.496466,0.000370,2.816105,0


In [70]:
pd.options.display.max_rows = 30
pd.options.display.min_rows = 30

## 주문 넣기와 거래 실행하기

In [72]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
pd.options.display.float_format = '{:.6f}'.format
pd.options.display.max_rows = 30
pd.options.display.min_rows = 30

In [73]:
client = Client(api_key=api_key, api_secret=secret_key, tld="com", testnet = True)

In [96]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units # NEW
        self.position = position # NEW
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades() # NEW!!!
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): # NEW!
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                print("GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                print("GOING NEUTRAL")
            self.position = 0

In [105]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1685533140272,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10039.72315765', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [89]:
# client.create_order(symbol="XRPUSDT", side="SELL", type="MARKET", quantity=155.5)

{'symbol': 'XRPUSDT',
 'orderId': 2203993,
 'orderListId': -1,
 'clientOrderId': 'o05oCY3xLvtH67BGi3kPoR',
 'transactTime': 1685532799059,
 'price': '0.00000000',
 'origQty': '155.50000000',
 'executedQty': '155.50000000',
 'cummulativeQuoteQty': '79.33610000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1685532799059,
 'fills': [{'price': '0.51020000',
   'qty': '155.50000000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 208126}],
 'selfTradePreventionMode': 'NONE'}

In [98]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]
units = 0.01
position = 0

In [99]:
trader = LongOnlyTrader(symbol, bar_length, return_thresh, volume_thresh, units, position)

In [100]:
trader.start_trading(historical_days=1/24)

........................GOING LONG
.............................................................................

In [104]:
trader.twm.stop()

.

CANCEL read_loop


## 거래 모니터링과 보고

In [101]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
pd.options.display.float_format = '{:.6f}'.format
pd.options.display.max_rows = 30
pd.options.display.min_rows = 30

..

In [102]:
client = Client(api_key=api_key, api_secret=secret_key, tld="com", testnet = True)

....GOING NEUTRAL
....

In [118]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 # NEW
        self.trade_values = [] # NEW
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0
    
    def report_trade(self, order, going):  # NEW
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, cum_profits))
        print(100 * "-" + "\n")

.....

In [107]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1685533140272,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10039.72315765', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [108]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]
units = 0.01
position = 0

In [109]:
trader = LongOnlyTrader(symbol, bar_length, return_thresh, volume_thresh, units, position)

In [110]:
trader.start_trading(historical_days=1/24)

......

----------------------------------------------------------------------------------------------------
2023-05-31 11:46:00.248000 | GOING LONG
2023-05-31 11:46:00.248000 | Base_Units = 0.01 | Quote_Units = 271.1003464 | Price = 27110.03464 
2023-05-31 11:46:00.248000 | Profit = 0 | CumProfits = 0.0 
----------------------------------------------------------------------------------------------------

........................................................

----------------------------------------------------------------------------------------------------
2023-05-31 11:48:00.286000 | GOING NEUTRAL
2023-05-31 11:48:00.286000 | Base_Units = 0.01 | Quote_Units = 271.1038 | Price = 27110.38 
2023-05-31 11:48:00.286000 | Profit = 0.003 | CumProfits = 0.003 
----------------------------------------------------------------------------------------------------

..........................................

In [111]:
trader.prepared_data.tail(10)

,Close,Volume,returns,vol_ch,position
Date,,,,,
2023-05-31 11:39:00,27115.530000,2.925531,0.000111,-0.192207,0
2023-05-31 11:40:00,27114.480000,1.018224,-0.000039,-1.055416,1
2023-05-31 11:41:00,27121.960000,5.663676,0.000276,1.716013,0
2023-05-31 11:42:00,27115.680000,2.311664,-0.000232,-0.896106,1
2023-05-31 11:43:00,27115.550000,0.233979,-0.000005,-2.290492,1
2023-05-31 11:44:00,27115.770000,0.081753,0.000008,-1.051529,0
2023-05-31 11:45:00,27110.020000,18.837680,-0.000212,NaN,1
2023-05-31 11:46:00,27098.070000,12.980960,-0.000441,-0.372375,1
2023-05-31 11:47:00,27110.380000,68.349060,0.000454,1.661144,0


................................................................

----------------------------------------------------------------------------------------------------
2023-05-31 11:52:00.382000 | GOING LONG
2023-05-31 11:52:00.382000 | Base_Units = 0.01 | Quote_Units = 271.009 | Price = 27100.9 
2023-05-31 11:52:00.382000 | Profit = 0 | CumProfits = 0.003 
----------------------------------------------------------------------------------------------------

........................

In [122]:
trader.trade_values

[-271.1003464, 271.1038, -271.009, 270.9765, -270.8779, 270.87779261]

In [123]:
np.sum(trader.trade_values)

-0.029153790000066238

In [114]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1685533920382,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.01000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9768.71761125', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

...............................

----------------------------------------------------------------------------------------------------
2023-05-31 11:55:00.054000 | GOING NEUTRAL
2023-05-31 11:55:00.054000 | Base_Units = 0.01 | Quote_Units = 270.9765 | Price = 27097.65 
2023-05-31 11:55:00.054000 | Profit = -0.033 | CumProfits = -0.029 
----------------------------------------------------------------------------------------------------

...........

In [121]:
trader.twm.stop()

.

CANCEL read_loop


## 거래 중에 자동 스탑 주문 설정

In [124]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
pd.options.display.float_format = '{:.6f}'.format
pd.options.display.max_rows = 30
pd.options.display.min_rows = 30

In [125]:
client = Client(api_key=api_key, api_secret=secret_key, tld="com", testnet = True)

In [126]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 # NEW
        self.trade_values = [] # NEW
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
        
        if event_time >= datetime(2023, 6, 2, 0, 0):
            self.twm.stop()
            if self.position != 0:
                order = client.create_order(symbol=self.symbol, side="SELL", type="MARKET", quantity=self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP")
                self.position = 0
            else:
                print("STOP")
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0
    
    def report_trade(self, order, going):  # NEW
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

In [127]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]
units = 0.01
position = 0

In [128]:
trader = LongOnlyTrader(symbol, bar_length, return_thresh, volume_thresh, units, position)

In [129]:
trader.start_trading(historical_days=1/24)

...........................................

In [130]:
trader.cum_profits

AttributeError: 'LongOnlyTrader' object has no attribute 'cum_profits'

.............

In [131]:
trader.trade_values

[]

....